In [1]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')



In [ ]:

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

In [3]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2922, done.
remote: Counting objects: 100% (2922/2922), done.
remote: Compressing objects: 100% (2452/2452), done.
remote: Total 2922 (delta 742), reused 1290 (delta 432), pack-reused 0
Receiving objects: 100% (2922/2922), 32.98 MiB | 26.80 MiB/s, done.
Resolving deltas: 100% (742/742), done.


In [4]:

%%bash
sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Reading package lists...
Building dependency tree...
Reading state information...
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1665514 sha256=368d2eed307a97375b0e2a41ab0cd67b28ab88bf22659d86a7f7df339612ae30
  Stored in directory: /tmp/pip-ephem-wheel-cache-14426vys/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22258 sha256=e1ad7f33b4f6834fb810589669d607987ee2448ec1dae8b6684a51170deec7f8
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-py3-none-any.whl size=43512 sha256=7fc5fafda0993b71c443609052a52431d432a61b240c0536c223870156bd5046
  Stored in directory: /r



  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
%%bash
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi/PythonAPI
make
cp -r pycocotools /content/models/research/

python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I../common -I/usr/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/

Cloning into 'cocoapi'...
../common/maskApi.c: In function ‘rleDecode’:
../common/maskApi.c:46:7: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
       ^~~
../common/maskApi.c:46:49: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
                                                 ^
../common/maskApi.c: In function ‘rleFrPoly’:
../common/maskApi.c:166:3: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
   for(j=0; j<k; j++) x[j]=(int)(scale*xy[j*2+0]+.5); x[k]=x[0];
   ^~~
../common/maskApi.c:166:54: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
   for(j=0; j<k; j++) x[j]=(int)(scale*xy[j*2+0]+.5); x[k]=x[0];
                                                      ^
../common/maskApi.c:167:3: warning: this ‘for’ clause does 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%ls

cocoapi/  drive/  models/  sample_data/


In [8]:
!mkdir workspace
!mkdir scripts

In [9]:
!mkdir workspace/training_demo

In [10]:
!mkdir workspace/training_demo/annotations
!mkdir workspace/training_demo/images
!mkdir workspace/training_demo/models
!mkdir workspace/training_demo/pre-trained-models
!mkdir scripts/preprocessing

In [11]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

In [12]:
%cp -r /content/drive/MyDrive/Colab_Notebooks/CV/L5/test /content/workspace/training_demo/images

In [13]:
%cp -r /content/drive/MyDrive/Colab_Notebooks/CV/L5/train /content/workspace/training_demo/images

In [14]:
%cp -r /content/drive/MyDrive/Colab_Notebooks/CV/L5/generate_tfrecord.py /content/scripts/preprocessing

In [15]:
with open('/content/workspace/training_demo/annotations/label_map.pbtxt', 'w') as f:
    f.write("item { id: 1 \n name: 'ship'}")

In [16]:
%cd scripts/preprocessing/
%ls

/content/scripts/preprocessing
generate_tfrecord.py


In [17]:
!python generate_tfrecord.py -x /content/workspace/training_demo/images/train -l /content/workspace/training_demo/annotations/label_map.pbtxt -o /content/workspace/training_demo/annotations/train.record

Successfully created the TFRecord file: /content/workspace/training_demo/annotations/train.record


In [18]:
!python generate_tfrecord.py -x /content/workspace/training_demo/images/test -l /content/workspace/training_demo/annotations/label_map.pbtxt -o /content/workspace/training_demo/annotations/test.record

Successfully created the TFRecord file: /content/workspace/training_demo/annotations/test.record


In [ ]:
%ls

generate_tfrecord.py


In [19]:
!mkdir /content/workspace/training_demo/models/my_ssd_mobilenet_v2_fpnlite

In [20]:
%cp -r /content/drive/MyDrive/Colab_Notebooks/CV/L5/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8 /content/workspace/training_demo/pre-trained-models/

In [ ]:
# %cp -r /content/drive/MyDrive/Colab_Notebooks/CV/L5/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8 /content/workspace/training_demo/models/my_ssd_mobilenet_v2_fpnlite

In [21]:
%cp /content/drive/MyDrive/Colab_Notebooks/CV/L5/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config /content/workspace/training_demo/models/my_ssd_mobilenet_v2_fpnlite

In [22]:
%cp /content/models/research/object_detection/model_main_tf2.py /content/workspace/training_demo

In [23]:
%cd ../../workspace/training_demo
%ls

/content/workspace/training_demo
annotations/  images/  model_main_tf2.py  models/  pre-trained-models/


In [ ]:
!python model_main_tf2.py --model_dir=/models/my_ssd_mobilenet_v2_fpnlite --pipeline_config_path=models/my_ssd_mobilenet_v2_fpnlite/pipeline.config

2021-09-22 17:51:24.938022: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-22 17:51:24.949855: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-22 17:51:24.950724: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-22 17:51:24.952619: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-22 17:51:24.953490: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S